## Read in a CSV and write out python code defining a data class for each row

In [50]:
import csv

def python_from_csv(filename, class_name):
    source_file = f"..\\django\\QuadEBdb\\tom\\generated_code\\{class_name}.py"
    indent = "    "

    header, first_row = list(csv.reader(open(filename)))[:2]

    with open(source_file, "w") as f:
        f.write("from django.db import models\n")
        f.write("\n")
        f.write(f"class {class_name}(models.Model):\n")
        for column_name, example_data in zip(header, first_row):
            try:
                _ = float(example_data)
                if example_data.count(".") > 0:
                    field_type = "FloatField(null=True)"
                else:
                    if len(example_data) > 8:
                        field_type = "BigIntegerField(null=True)"
                    else:
                        field_type = "IntegerField(null=True)"
            except:
                field_type = 'CharField(max_length=100, null=True)'
            if column_name.lower() == "id":
                #django creates a column named id as the primary key, so we can't also use it here
                column_name = "Identifier"
            f.write(f"{indent}{column_name.replace(" ", "_")} = models.{field_type}\n")

# python_from_csv("TICv8 Targets.csv", "Tess_TICv8")
# python_from_csv("GAIA DR2 Example.csv", "Gaia_DR2")
# python_from_csv("GAIA Neighbors.csv", "Gaia_DR3")
python_from_csv("2MASS.csv", "TwoMass")
